In [2]:
print('hello world')
import pandas as pd
import numpy as np
import shutil
import itertools
from collections import namedtuple
from glob import glob
# import pandas_profiling
from datetime import timedelta, datetime
# pd.options.display.float_format = '${:,.0f}'.format
pd.options.display.float_format = '{:,.0f}'.format
import os
print(os.getcwd())
os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
# os.chdir('C:/Users/hansoc10/Desktop/Programming/Python/datafiles/')
print(os.getcwd())
# pd.show_versions()
print(pd.__version__)
np.__version__

hello world
J:\LEGAL\share1\IP_forecasting_test_env
J:\LEGAL\share1\IP_forecasting_test_env
0.24.2


'1.16.3'

In [ ]:
'''
load the current rolled up forecast into a dataframe
create list of new or updated files containing forecasts to process
concatenate the forecasts in those files into a dataframe
concatenate the new forecasts into rollup forecast (known as current forecast)
move processed forecasts to the processed file for safekeeping
write updated forecast back to current file
'''

'''
create a dataframe of the current master rolledup forecast
'''
current_forecast_MDT = pd.read_excel('current_forecast\MDT_current_forecast_rollup.xlsx')

'''
create a list of filenames of the new forecasts that are in the current file folder
'''
files_to_process = sorted(glob('CAH_forecast*.xlsx'))
print(files_to_process)

'''
create a dataframe from the files to process by concatentating them into a single dataframe and
give them a timestamp so we know when they were processed
'''
new_forecasts_to_process = pd.concat((pd.read_excel(file) for file in files_to_process), ignore_index=True)
new_forecasts_to_process['timestamp'] = pd.Timestamp.today()

'''
add the new forecasts to the current master rolled up forecast
'''
updated_forecast = pd.concat([current_forecast_MDT, new_forecasts_to_process], ignore_index=True, sort=False)

'''
move the forecasts that were the new to be forecasts that are now processed to a processed file folder
so we don't process the forecasts twice
'''
destination = "submitted_forecast_processed/"
for files in files_to_process:
    if files.endswith(".xlsx"):
        shutil.move(files, destination)

'''
save the now updated roll up forecast back to its original spot
'''
updated_forecast.to_excel('current_forecast/MDT_current_forecast_rollup.xlsx', index=False)
updated_forecast.head()

In [ ]:
'''
create list of new forecasts this may be the one that works best
'''
files_to_process = sorted(glob('*CAH_forecast*.xlsx'))
print(files_to_process[0])
# df = pd.read_excel(files_to_process[0])
# df.head()

for file in files_to_process:
    df = pd.read_excel(file)
    df['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR')
    # print(df.head())
    df['month'] = df.task_date.dt.month
    df['year'] = df.task_date.dt.year
    df['day'] = df.task_date.dt.day
    df['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR').quarter
    df.loc[((df.task_date.dt.month >= 5) | ((df.task_date.dt.month == 4) & (df['fiscal_qtr'] == 1))), 'fiscal_yr'] = df.task_date.dt.year + 1
    df.loc[((df.task_date.dt.month <= 4) & (df['fiscal_qtr'] != 1)), 'fiscal_yr'] = df.task_date.dt.year
    df.to_excel(file, index=False)

df.head()
    
# df = pd.read_excel([file for file in files_to_process])
# df

In [ ]:
'''
Make dataframe with the current master forecast
I moved this to jupyter notebook MDT forecast FINAL on 10/13/2019
'''
current_forecast_MDT = pd.read_excel('current_forecast\MDT_current_forecast_rollup.xlsx')


'''
create list of new forecasts
'''
files_to_process = sorted(glob('*CAH_forecast*.xlsx'))
print(files_to_process)

'''
for all the forecasts in the list of files to process
create a single dataframe of the new forecasts by concatenating them into a single dataframe
give them a timestamp so we know when it occurred as this will allows us to see all hitorical forecasts
create or update the forecast status to current
'''


new_forecasts_to_process = pd.concat((pd.read_excel(file) for file in files_to_process), ignore_index=True, sort=False)
new_forecasts_to_process['timestamp'] = pd.Timestamp.today()
new_forecasts_to_process['frcst_status'] = 'current'


# print(new_forecasts_to_process.columns)


'''
create a list of features in the new, aka, current, forecasts that can distiguish these new or updated forecasts 
from forecasts that are current forecasts in the current master forecast.
These lists will be used to change the status of the previous current forecast to obsolete
because these new forecasts are to be considered the current forecast.
'''
frmnme = new_forecasts_to_process['firm_name'].unique()
mtrnum = new_forecasts_to_process['matter_num'].unique()
# print(frmnme)
# print(mtrnum)

'''
The attorneys and firms are asked to fill out the task_date column for their forecasts
using that date, this code updates a number of series columns that 
make it easy to slice and dice the forecast.
Importantly, this does it so that the MDT fiscal year is used exactly.
MDT is a 5253 type fiscal year and the way the programming works is to match the last friday of the quarter
that there is a step to use a day offset of 1 to make sure we get it right for the fiscal year to the day
and this last step is necessary because the fiscal offsets for when done for the quarter diesn't get the fiscal year correct
'''

new_forecasts_to_process['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in new_forecasts_to_process.task_date], freq='Q-APR')
new_forecasts_to_process['month'] = new_forecasts_to_process.task_date.dt.month
new_forecasts_to_process['year'] = new_forecasts_to_process.task_date.dt.year
new_forecasts_to_process['day'] = new_forecasts_to_process.task_date.dt.day
new_forecasts_to_process['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in new_forecasts_to_process.task_date], freq='Q-APR').quarter

new_forecasts_to_process.loc[((new_forecasts_to_process.task_date.dt.month >= 5) | ((new_forecasts_to_process.task_date.dt.month == 4) & (new_forecasts_to_process['fiscal_qtr'] == 1))), 'fiscal_yr'] = new_forecasts_to_process.task_date.dt.year + 1
new_forecasts_to_process.loc[((new_forecasts_to_process.task_date.dt.month <= 4) & (new_forecasts_to_process['fiscal_qtr'] != 1)), 'fiscal_yr'] = new_forecasts_to_process.task_date.dt.year



'''
see if any of the new forecasts to process for a given firm and a resulting matter
are already in the master database as a current forecast
if so, change the current forecast frcst status in the master database
from "current" to "obsolete".
then late in the program the new_forecasts_to_process will be added as current forecasts
to do this identify the rows by firm name and matter number and put in variable rows
for those rows change forecast status to obsolete to prepare for the new forecasts
'''

# need to fill in

'''
add new forecasts to the current forecast creating an updated master forecast
'''
updated_forecast = pd.concat([current_forecast_MDT, new_forecasts_to_process], ignore_index=True, sort=False)

'''
write the newly created current forecast back to excel.
'''
updated_forecast.to_excel('current_forecast/MDT_current_forecast_rollup.xlsx', index=False)

'''
adding timestamp to files_to_process
'''
timestmp_prefix = (str(pd.Timestamp.today().year)
                   + str(pd.Timestamp.today().month)
                   + str(pd.Timestamp.today().day)
                   + str(pd.Timestamp.today().hour) 
                   + str(pd.Timestamp.today().minute))

'''
i read the following list comprehension as taking file f and formatting the ouput name 
to be the prefix followed by the original file name
for all files in the directory meeting the filename criteria in the glob function above
'''
[os.rename(f, "{}{}".format(timestmp_prefix, f)) for f in files_to_process]

# '''
# move the forecasts that were the new to be forecasts 
# that are now processed and have a timestamp prefix added to their name 
# to a processed file folder "submitted_forecast_processed"
# so we don't process the forecasts twice
# '''
# destination = "submitted_forecast_processed/"
# for files in files_to_process:
#     if files.endswith(".xlsx"):
#         shutil.move(files, destination)



# test_df = new_forecasts_to_process[['task','firm_name', 'matter_num', 'timestamp', 'frcst_status']]
test_df = current_forecast_MDT[['task','firm_name', 'matter_num', 'timestamp', 'frcst_status']]

# print(test_df)
# updated_forecast
new_forecasts_to_process.head()

In [ ]:
#test_df.head()
current_forecast_MDT.head()

In [ ]:
rows = (test_df['firm_name'] == frmnme[0]) & (test_df['matter_num'] == mtrnum[0]) & (test_df['frcst_status'] == 'current')
# print(rows)
test_df.loc[rows, ['frcst_status']] = ['obsolete']
# print(frmnme[0])
test_df.head()

In [ ]:
for num in range(len(frmnme)):
    print(num)
    rows = (test_df['firm_name'] == frmnme[num]) & (test_df['matter_num'] == mtrnum[num]) & (test_df['frcst_status'] == 'current')
    test_df.loc[rows, ['frcst_status']] = ['obsolete']

test_df

In [ ]:
df = pd.read_excel('CAH_forecast_Test__55_testdata.xlsx')
# df = pd.read_excel('test_result.xlsx')
pine = itertools.zip_longest(df['firm_name'], df['matter_num'], fillvalue='whats_up')
bubba = list(pine)

new_forecast_identifier = []
for item in bubba:
    print(item)
    if item not in new_forecast_identifier:
        new_forecast_identifier.append(item)
print(new_forecast_identifier)

name = [x[0] for x in new_forecast_identifier]
mnum = [x[1] for x in new_forecast_identifier]

for num in range(len(name)):
    rows = (df['firm_name'] == name[num]) & (df['matter_num'] == mnum[num]) & (df['frcst_status'] != 'current')
    df.loc[rows, ['frcst_status']] = ['obsolete']
    print('chadd')
for num in range(len(name)):
    print(name[num])

df.to_excel('test_result.xlsx')


In [ ]:
'''
this is the one that works best!!
'''
# make database
df = pd.read_excel('CAH_forecast_Test__55_testdata.xlsx')

# create a row identifier by zipping two columns/series together
zipped_clms = itertools.zip_longest(df['firm_name'], df['matter_num'], fillvalue='whats_up')

# make a list of tuples of the unique row identifiers
new_forecast_identifier = []
for item in zipped_clms:
    if item not in new_forecast_identifier:
        new_forecast_identifier.append(item)

# use list comprehension to make a list of the unique firm name matter number combinations
# and use the list to set frcst_status
frmnme = [x[0] for x in new_forecast_identifier] # firm name
mtrnum = [x[1] for x in new_forecast_identifier] # matter number
print(frmnme)

for num in range(len(frmnme)):
    print(frmnme[num], mtrnum[num])
    rows = (df['firm_name'] == frmnme[num]) & (df['matter_num'] == mtrnum[num]) & (df['frcst_status'] != 'current')
    df.loc[rows, ['frcst_status']] = ['obsolete']

df.to_excel('test_result.xlsx', index=False)
df.head()

In [ ]:
df = pd.read_excel('CAH_forecast_Test__55_testdata.xlsx')
# df = pd.read_excel('test_result.xlsx')
pine = itertools.zip_longest(df['firm_name'], df['matter_num'], fillvalue='whats_up')
bubba = list(pine)
print(bubba)
print([x[0] for x in bubba])
[x[1] for x in bubba]



In [ ]:
forecast_id = namedtuple('ID', ['name', 'number'])
s = forecast_id('chad', 2345)
print(s.name)
print(s.number)
s

In [ ]:
df = pd.DataFrame([[1, 2], [4, 5], [5, 5], [5, 9], [55, 55]], columns=['max_speed', 'shield'])
df['frcst_stus'] = 'current'
df

In [ ]:
# df.loc[:, ['max_speed', 'frcst_stus']]

In [ ]:
# df.loc[:, ['max_speed', 'frcst_stus']]
# df

In [ ]:
rows = (df['shield'] == 5) & (df['max_speed'] == 5) & (df['frcst_stus'] == 'current')
print(type(rows))
print(rows)

m = (df['shield'] == 5) & (df['max_speed'] == 5) & (df['frcst_stus'] == 'current')
df.loc[rows, ['max_speed', 'frcst_stus']] = [100, 'hello']
df


In [ ]:
'''
list comprehension is output expression for item in iterable if condition
'''
numbers = [1, 2, 3, 4, 5]
squares = [number**2 for number in numbers]
print(squares)

In [ ]:
'''
medtronic fiscal quarters
'''

# list of dates - test data
first_date = datetime(2018, 1, 1)
last_date = datetime(2018, 5, 20)
x = 1
list_of_dates = [date for date in np.arange(first_date, last_date, timedelta(days=x)).astype(datetime)]


# create the dataframe
df = pd.DataFrame({'dates': list_of_dates})

# df['Qdate'] = [date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.QuarterEnd() for date in  df.dates]
df['Q_end_date'] = [date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.dates]

# df['fscl_Qtr'] = pd.PeriodIndex(df.dates, freq='Q-APR')  
df['fscl_Qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.dates], freq='Q-APR')

df.head()


In [ ]:
# df = pd.read_excel('testing_data_forfscl_qtr.xlsx')  Copy of CAH_forecast_Teleflex_20190826 - KJN Setup rev_CAH_forecast_MASTER_20191001
df = pd.read_excel('CAH_forecast_robins_titan.xlsx')

df['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR')
df['month'] = df.task_date.dt.month
df['year'] = df.task_date.dt.year
df['day'] = df.task_date.dt.day
df['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.task_date], freq='Q-APR').quarter

df.loc[((df.task_date.dt.month >= 5) | ((df.task_date.dt.month == 4) & (df['fiscal_qtr'] == 1))), 'fiscal_yr'] = df.task_date.dt.year + 1
df.loc[((df.task_date.dt.month <= 4) & (df['fiscal_qtr'] != 1)), 'fiscal_yr'] = df.task_date.dt.year


df.head(10)
df.to_excel('CAH_forecast_Titan_Robins.xlsx', index=False)
df

In [ ]:
'''
medtronic fiscal quarters
'''

# list of dates - test data
first_date = datetime(2017, 12, 21)
last_date = datetime(2018, 5, 20)
x = 2
list_of_dates = [date for date in np.arange(first_date, last_date, timedelta(days=x)).astype(datetime)]


# create the dataframe
df = pd.DataFrame({'dates': list_of_dates})

# no work right df['Qdate34'] = [date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.QuarterEnd() for date in  df.dates]
df['Q_end_date'] = [date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.dates]

# df['fscl_Qtr44'] = pd.PeriodIndex(df.dates, freq='Q-APR')  # not accurate
df['fscl_YrQtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.dates], freq='Q-APR')
df['fscl_year_quarter'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.dates], freq='Q-APR')
df['month'] = df.dates.dt.month
df['year'] = df.dates.dt.year
df['day'] = df.dates.dt.day
df['fscl_days_of_period'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.dates], freq='Q-APR').day
df['fscl_quarter_num'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.dates], freq='Q-APR').quarter
df['fscl_year_huh'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in  df.dates], freq='Q-APR').year

df.iloc[5:25]
df.iloc[5:25].to_excel('illustrate fiscal info.xlsx')

In [ ]:
'''
adding timestamp to files names
'''
# tday = pd.Timestamp.today()

timestmp_prefix = (str(pd.Timestamp.today().year)
                   + str(pd.Timestamp.today().month)
                   + str(pd.Timestamp.today().day)
                   + str(pd.Timestamp.today().hour) 
                   + str(pd.Timestamp.today().minute))
'''
i read the following list comprehension as taking file f and formatting the ouput name 
to be the prefix followed by the original file name
for all files in the directory meeting the filename criteris in the glob function

'''
[os.rename(f, "{}{}".format(timestmp_prefix, f)) for f in glob("*file_for_timstamp_prefix*.csv")]
timestmp_prefix
# print(files_to_process)
# [os.rename(f, "{}{}".format(timestmp_prefix, f)) for f in files_to_process]


In [ ]:
frmnme = new_forecasts_to_process['firm_name'].unique()
mtrnum = new_forecasts_to_process['matter_num'].unique()
# print(frmnme)
# print(mtrnum)

In [ ]:
df = pd.read_excel('test_passport_data_utf8_back_excel.xlsx')


# df = pd.read_excel('passport_data.xlsx')
# df = pd.read_csv('passport_data.xlsx', encoding='latin1')
df

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 14 16:07:02 2019

@author: hansoc10

This is from my spyder file
"""
import itertools
import csv
import datetime
# import xlrd
# import openpyxl
import time  # from epoch 1/1/1970
import os
import pytz
import re


"""
MATTER MANAGEMENT PROGRAM FOR MANAGING MATTER SPEND
"""


"""
GLOBAL VARIABLES
"""
accrual_date_key = 'Accrual Date'
current_time = datetime.datetime.now(tz=pytz.utc).astimezone(pytz.timezone('US/Central'))


"""
CLASSES AND ASSOCIATED METHODS
"""


class Matter:
    """
    Matter class is the number and status of a matter.
    """
    def __init__(self, number):
        self.number = number


class Matter_w_Name(Matter):
    """
    Add caption and nickname (common name) to matter.
    """
    def __init__(self, number, caption, nickname):
        super().__init__(number)
        # another way is to use Matter.__init__(etc)
        self.caption = caption
        self.nickname = nickname

    def close_matter(self, status):
        if status == "Open":
            self.status = "Open"
        if status == "Closed":
            self.status = "Closed"
        elif status != "Closed":
            self.status = "Open"
            print('Matter remains open.  You must send \"Closed\" to close the matter.')
        return self.status


class AOP(Matter):
    """
    To get the whole AOP you can do this: print(My_AOP.AOP)
    """
    def __init__(self, number, fscl_year, fq1, fq2, fq3, fq4):
        super().__init__(number)
        self.AOP = [number, fscl_year, fq1, fq2, fq3, fq4]
        self.number = number
        self.fiscal_year = fscl_year
        self.FQ1 = fq1
        self.FQ2 = fq2
        self.FQ3 = fq3
        self.FQ4 = fq4

        @classmethod
        # This method gets the AOP for a matter.  AOP.from_screen().abs
        def from_screen(cls):
            print(
                'Enter AOP for a litigation matter in the format noted below.'
            )
            print('\t   Mattter#, FYR, $Q1$, $Q2$, $Q3$, $Q4$.')
            print('Example:     23322, 2019, 3000, 4000, 5050, 6000')
            number, fscl_year, fq1, fq2, fq3, fq4 = input(
                "Enter AOP:  ").split()
            return cls(number, fscl_year, fq1, fq2, fq3, fq4)

        @classmethod
        # Placeholder This method gets the AOP for a matter.
        # To use it 'new_aop = AOP.from_file()'
        def from_file(cls):
            pass


class Forecast(Matter):
    """
    To get the whole Forecast you can do this: print(variable_name.Forecast)
    Forecast is planned spend.
    It requires
    (1) the matter number to which the forecast applies,
    (2) provides the fiscal quarter that the
    first planned quarterly spend applies to, and
    (3) spend broken out by quarters.
    """
    def __init__(self, number, start_fscl_qtr, *spend):
        super().__init__(number)
        self.FCST = [number, start_fscl_qtr, *spend]
        self.number = number
        self.quarter_start = start_fscl_qtr
        self.spend = spend


class Actual(Matter):
    """
    This is the actual spend.
    It requires the matter number.
    It requres the fiscal quarter for the start of the spend.
    It takes as many quarters of spend as you want.
    """

    def __init__(self, number, start_fscl_qtr, *spend):
        super().__init__(number)
        self.ACTUAL = [number, start_fscl_qtr, *spend]
        self.number = number
        self.quarter_start = start_fscl_qtr
        self.spend = spend


"""
FUNCTIONS
"""


def is_number_true(s):
    """
    Returns True if string is a number.
    """
    try:
        float(s)
        return True
    except ValueError:
        return False


def timestamp():
    """
    Unadjusted epoch time to create a timestamp integer.
    """
    return round(time.time())


def fiscal_quarter_clculatr(month, year):
    """
    Takes a calendar month and four-digit calendar year
    to create a string representing MDT fiscal quarters.
    """
    b = slice(2, 5)
    Year = str(year)
    Year = Year[b]
    Year_plusone = str(year + 1)
    Year_plusone = Year_plusone[b]
    if month == 5 or month == 6 or month == 7:
        FSCLQTR = 'FY' + Year_plusone + 'Q1'
    elif month == 8 or month == 9 or month == 10:
        FSCLQTR = 'FY' + Year_plusone + 'Q2'
    elif month == 11 or month == 12:
        FSCLQTR = 'FY' + Year_plusone + 'Q3'
    elif month == 1:
        FSCLQTR = 'FY' + Year + 'Q3'
    elif month == 2 or month == 3 or month == 4:
        FSCLQTR = 'FY' + Year + 'Q4'
    return FSCLQTR


def AOP_fsclqtr_str(year):
    """
    use two-digit or four-digit int of fiscal year
    to create AOP fiscal quarter strings
    """
    aop_list = ["Matter Number", "AOP Fiscal Year"]
    yr = str(year)
    if len(yr) == 4:
        b = slice(2, 5)
        yr = yr[b]
    elif len(str(year)) == 2:
        yr = year
    elif len(str(year)) != 2 and len(yr) != 4:
        print("There is something wrong with the fiscal year."
              "It must be either two digits, e.g., 19, "
              "or four digits, e.g. 2019")
    cntr = itertools.count(start=1, step=1)
    for step in range(4):
        aop_list.append(str("FY" + yr + "Q" + str(next(cntr))))
    return aop_list


def FCST_fsclqtr_str(qtr_start, total_qtrs):
    """
    takes a string that represents the starting fiscal quarter
    for a forecast and the total number of quarters in the forecast
    (this is sent via len(total spend breakout) to create
    forecast list that has the matter number, fiscal quarter start,
    and an ordered list of the fiascal quarters that represent the spend
    in the forecasted quarters no matter how long the forecast goes:
        'Matter Number', 'Fiscal Quarter Start', 'FY##Q#' ...
    """

    fcst_list = ["Matter Number", "Fiscal Quarter Start"]  # in clean up remove this from the function
    fcst_qrtrs = total_qtrs
    b = slice(2, 4)
    c = slice(5, 6)
    fisc_yr = qtr_start[b]
    fisc_yr_qtr_start = int(qtr_start[c])

    for step in range(fisc_yr_qtr_start, 5):
        fcst_list.append(str("FY" + fisc_yr + "Q" + str(step)))

    remaining_qrtrs = fcst_qrtrs - step
    while remaining_qrtrs > 0:
        next_fscl_yr = int(fisc_yr) + 1
        fisc_yr = str(next_fscl_yr)
        for step in range(1, 5):
            fcst_list.append(str("FY" + fisc_yr + "Q" + str(step)))
            remaining_qrtrs -= 1
    return fcst_list


def remove_comma_from_string(string):
    """
    take string apply replace method to remove commas
    """
    no_comma_string = string.replace(',', '')
    return no_comma_string


def convert_sign_of_num(number):
    """
    used to convert Passport spend to a positive number
    but simply converts pos to neg and neg to pos
    """
    opp_sign_number = number * -1
    return opp_sign_number


def get_a_month_year_int(prompt, num_type=None):
    """
    USED TO GET SOMEONE TO ENTER A POSITIVE MONTH AND A YEAR.
    THIS FUNCTION REQUIRES THE USE OF THE CURRENT YEAR.
    """
    while True:
        try:
            value = int(input(prompt))
        except ValueError:
            print("enter a positive, whole number")
            continue
        if value <= 0:
            print("enter a positive, whole number.")
            continue
        if num_type is 'month' and value > 12:
            print("Month must not exceed 12.")
            continue
        if num_type is 'year' and value < current_time.year:
            print("year must be >= current year.")
            continue
        if num_type is 'year' and value > current_time.year + 5:
            print(" Let's not estimate this far into the future")
            continue
        else:
            break
    return value


def merge(a, b):
    """
    Merge two lists.
    Intended use create ordered list of fiscal quarter strings.
    """
    max_offset = len(b)
    for i in reversed(range(max_offset + 1)):
        # checks for equivalence of decreasing sized slices
        if a[-i:] == b[:i]:
            break
    return a + b[i:]


def row_is_empty(row):
    """
    Joins a row that is a list (usually)
    If row cotains nothinf returns true
    """
    if len("".join(row)) == 0:
        return True
    else:
        return False


def row_contains_my_flag(flag_str, row):
    """
    joins a row and checks it for a string that we are trying to flag
    returns true if it finds the flag.
    """
    if re.search(flag_str, ("".join(row)), flags=re.I) is not None:
        return True
    else:
        return False


def date_pull(str):
    """
    takes a string with a date at the end of the string.
    extracts and returns the date
    This might give an error if there is no date.
    Need to work in error handling
    """
    pulled_date = re.findall(r'[\d./]+$', str)
    # print(type(pulled_date[0]), 'this is pulled date')
    return pulled_date[0]


"""
EXECUTIONS
"""
os.chdir('C:/Users/hansoc10/Desktop/Programming/Python/datafiles/')
# print(os.getcwd())
# print(os.listdir())

"""
list_of_files = ['actual_input.txt.txt',
    'Covidien v. Esch Forecast Revision_Robins Kaplan_9.7.2018.csv',
    'Covidien v. Esch Robins Kaplan LLP_Medtronic Forecast Revision 1.2.2019 (004).csv',
    'page1 Williams  Connolly - Medtronic Accruals and Unpaid through 12 17 2018_.csv',
    'page2 Williams  Connolly - Medtronic Accruals and Unpaid through 12 17 2018_.csv',
    'Williams  Connolly - Medtronic Accruals and Unpaid through 12 17 2018_.xlsm',
    'Winston Strawn Forecast_New Forecast_Heather Lecce.csv']
"""

print('creating a matter and adding a nickname')
My_Matter = Matter(5569)

My_Matter = Matter_w_Name(5569, "Sassole v. MDT", "Sasso")
# print(My_Matter.number)
# print(My_Matter.caption)
# print(My_Matter.nickname)
# print(My_Matter.number)
# print('end of the created matter. \n', )

print('creating an AOP')
My_AOP = AOP(5569, 2018, 23, 24, 56, 67)
# print(My_AOP.number)
print('this is my AOP:', My_AOP.AOP)
# print(My_AOP.AOP[2])
# print(My_AOP.fiscal_year)
# print(My_AOP.FQ1)

print('creating an AOP list with the fiscal quarter strings')
zipped_aop = list(zip(AOP_fsclqtr_str(My_AOP.fiscal_year), My_AOP.AOP))

print('construct fiscal quarter from m-y')
FQtr = fiscal_quarter_clculatr(11, 2019)
print('e.g., 11, 2019 gives the following: ', FQtr, '\n')

My_Forecast = Forecast(5569, "FY19Q1", 234, 56756, 456, 45687, 444,
                       6565, 2345, 3456, 4576, 45765, 45764, 45765, 2345)
print('Below are parts of My_Forecast')
# print(type(My_Forecast))
print(My_Forecast.FCST)
# print(My_Forecast.quarter_start)
# print(My_Forecast.spend)
# print(My_Forecast.number)
# print(My_Forecast)
print('end of forecast \n')

print('creating a forecast list with the fiscal quarter strings')

My_fcst_string = FCST_fsclqtr_str(My_Forecast.quarter_start, len(My_Forecast.spend))

zipped_forecast = list(zip(My_fcst_string, My_Forecast.FCST))
# print(zipped_forecast, 'this was zipped')

zipped_long_forecast = list(itertools.zip_longest(My_fcst_string, My_Forecast.FCST))
# print(zipped_forecast, 'this was zipped_longest \n')
# print('end of forecast listing')

print('gathering actual spend to compare to forecasts and AOPs')
"""
will eventually make this so I can iterate over the matters
"""

read_file = open('actual_input.txt', 'r')
# actual imput is just a text file of the Chaz_
actual_list = []
for line in read_file:
    if 'FY1' in line.upper():
        if 'Total' not in line:
            pattern = re.compile(r'FY')
            matches = pattern.finditer(line)
            pattern_2 = re.compile(r'\$+\d')
            matches_2 = pattern_2.finditer(line)

            for match in matches_2:
                # print(line[0:7])
                qtr = line[0:7].replace('-', '')
                # print(qtr, "chadqtr")
                spend_line = line[9:None]
                # print(spend_line, "ship")
                spend_line_1 = re.sub(r'.*?\$', '', spend_line)
                # print(spend_line_1, 'shape')
                spend = spend_line_1.replace('\n', '')
                # print(spend)
                actual_list.append(qtr)
                actual_list.append(spend)
read_file.close()

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
for i in range(len(actual_list)):
    print(actual_list[i])
print('chad test:', actual_list[0], actual_list[1])

Actual_Spend = []
for item in range(1, len(actual_list), 2):
    Actual_Spend.append(int(actual_list[item]))
    print(Actual_Spend, 'actual spend')

Actual_Prep = [5569]
Actual_Prep.append(actual_list[0])
Actual_Prep.extend(Actual_Spend)
print(Actual_Prep, 'actual prep')

My_Actual = Actual(5569, "FY19Q3")
My_Actual.ACTUAL = Actual_Prep
print(My_Actual.ACTUAL, 'chazzzzyy')


"""
Merging lists
"""
tester_AOP = AOP_fsclqtr_str(My_AOP.fiscal_year)
tester_1 = tester_AOP.remove('Matter Number')
tester_2 = tester_AOP.remove('AOP Fiscal Year')
print('this is tester tester_AOP:', tester_AOP)

tester_FCST = My_fcst_string
tester_FCST.remove('Matter Number')
tester_FCST.remove('Fiscal Quarter Start')
tester_FCST.insert(0, 'FY18Q4')
print('chizz', tester_FCST)


# print('this is tester chupp for fcst:', tester_FCST)
merged_fiscal_quarters = merge(tester_AOP, My_fcst_string)
print('merged fiscal quarters:', merged_fiscal_quarters)

"""
END MERGING
"""

slicer = slice(1, 6, 2)
check = merged_fiscal_quarters[slicer]
print(check, 'this is check')


f = open('AOP_output.txt', 'a')
f.write(str(timestamp()))
f.write(',')
f.write(str(My_AOP.AOP))
# f.write('\t')
# f.write('what is going on?')
f.write('\n')
f.close()

print('INVOICED ACCRUALS ARE BELOW')
with open('invoicedAccruals.csv', 'r') as csv_file:
    """
    Step over the first few (5 in this case) rows of accrual spreadsheet
    This gets you to the fieldnames row
    Reader is an iterator that iterates over rows in cvs file.
    """
    reader = csv.reader(csv_file)
    with open('invoiced_accruals_output.csv', 'w', newline='') as csv_out_file:
            for step in range(5):
                """
                This corresponds to the existing accrual format used by MDT.
                """
                header = next(reader)
                # print(header)
                if step == 2:
                    accrual_date_value = date_pull(header[1])
                    print(accrual_date_value, 'accrual date value')
            header.insert(0, accrual_date_key)
            # print(header, 'header chad')
            Dictwriter = csv.DictWriter(csv_out_file,
                                        fieldnames=header,
                                        delimiter=',')
            Dictwriter.writeheader()
            # print(header, 'HIPPYhippy')
            for row in reader:
                if row_is_empty(row) is True:
                    continue
                if row_contains_my_flag('total', row):
                    continue
                if len(row[0] + row[1] + row[2] + row[3] + row[4] +
                       row[5] + row[6] + row[7]) == 0:
                    continue
                if len(row[1]) != 0:    # this fills in missing information
                    row_backup = row.copy()
                elif len(row[1]) == 0:
                    row[0] = row_backup[0]
                    row[1] = row_backup[1]
                    row[6] = row_backup[6]
                    row[7] = row_backup[7]
                for i in range(4, 6, 1):
                    """
                    This code section extracts number strings by
                    removing commas;
                    removing dollar signs; and
                    replaces them with a decimal number, a float.
                    Applied to rows 2-5, which are the attorney fee cells
                    in the spreadsheet.
                    """
                    num_extract = re.sub(r'[$, ]+', "", row[i])
                    row[i] = num_extract
                    # print(row[i], type(row[i]))
                row.insert(0, accrual_date_value)  # from a date above header.
                invoiced_accrual_data = dict(zip(header, row))
                Dictwriter.writerow(invoiced_accrual_data)

print('\n\n UNBILLED ACCRUALS ARE BELOW')
# f = open('invoiced_accruals_output.txt', 'w')
# test_from_excel.csv unbilledAccruals.csv
with open('unbilledAccruals.csv', 'r') as csv_file:
    """
    reader is an iterator that iterates over rows in cv file.
    So you could read reader as row.
    """
    reader = csv.reader(csv_file)
    # unbilled_accruals_output.csv
    with open('unbilled_accruals_output.csv', 'w', newline='') as csv_out_file:
#     reader = csv.reader(csv_file)
        for step in range(5):
            header = next(reader)
            if step == 2:
                # print(header[1], type(header[1]))
                accrual_date = date_pull(header[1])
                # print(step, accrual_date, 'chipster')
        header.insert(0, accrual_date_key)
        Dictwriter = csv.DictWriter(csv_out_file,
                                    fieldnames=header,
                                    delimiter=',')
        Dictwriter.writeheader()
        for row in reader:
            if row_is_empty(row) is True:
                continue
            if row_contains_my_flag('total', row):
                continue
            if len(row[1]) != 0:
                row_backup = row.copy()
            elif len(row[1]) == 0:
                """
                This section detects a row that depends from a row before it.
                Want all information to be in the row.
                Copy certain data to fill in cells
                Row_backup that has a complete set of data.
                """
                row[0] = row_backup[0]
                row[1] = row_backup[1]
                row[6] = row_backup[6]
                row[7] = row_backup[7]
            for i in range(2, 6, 1):
                """
                This code section extracts number strings removing commas;
                removing dollar signs; and replaces it with a decimal number.
                It works on columns 2-5, which are the fees cells in the spreadsheet.
                """
                # print('row i = ', row[i])
                num_extract = re.sub(r'[$, -]+', "", row[i])
                # print('num extract =', num_extract)
                if num_extract == "":
                    num_extract = 0.0
                    # print(num_extract, 'howdy')
                # print(num_extract)
                # dash_replace = re.sub(r'[-]+', "0", row[i])
                row[i] = num_extract
                if row[7] == "":
                    row[7] = 'No one Identified'
            row.insert(0, accrual_date)
            unbilled_accrual_data = dict(zip(header, row))
            Dictwriter.writerow(unbilled_accrual_data)

x = is_number_true(-345567.67)
print(x)



"""
Actual data calculated for quarter below
"""
print('ACTUAL DATA BELOW HERE')


"""
with open('chad_tt_data.csv', 'a') as csv_file:
    reader = csv.DictReader(islice(csv_file, 4, None))
    spend_dict = {}
    #print(type(spend_dict))
    print(spend_dict.keys(), "once?")
    for row in reader:
        print(row)




THIS IS THE BEGINNING OF BUILDING A FORECAST FOR A MATTER FROM THE GROUND UP

with open('chad_tsk_mngr_input.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for line in csv_reader:
        f = open('test_file.txt', 'a')
        print('Next task:', line[0])
        tsk_month = get_a_month_year_int(
            "Enter Month of Task: ", num_type='month')
        tsk_year = get_a_month_year_int(
            "Enter Year of Task: ", num_type='year')
        tsk_day = 1
        tsk_date = str(tsk_month) + "/1/" + str(tsk_year)
        tsk_date2 = datetime.date(tsk_year, tsk_month, tsk_day)
        print(tsk_date)
        print(tsk_date2)
        f.write(tsk_date)
        f.write('\t')
"""

In [ ]:
'''
from pythonista
'''
while True:
    try:
        age = int(input("Please enter your age: ")) 
    except ValueError:
        print("Sorry, I didn't understand that.") #better try again... Return to the start of the loop 
        continue
    else:
        break
            
if age >= 18:
    print("You are able to vote in the United States!")
else:
    print("You are not able to vote in the United States.") # another 

    
while True:
    data = input("Please enter a loud message (must be all caps): ")
    if not data.isupper():
        print("Sorry, your response was not loud enough.")
        continue
    else:
        break

while True:
    data = input("Pick an answer from A to D:") 
    if data.lower() not in ('a', 'b', 'c', 'd'):
        print("Not an appropriate choice.")
    else:
        break
#another

while True:
    try:
        age = int(input("Please enter your age: ")) 
    except ValueError:
        print("Sorry, I didn't understand that.") 
        continue    
    if age < 0:
        print("Sorry, your response must not be negative.")
        continue
    else:
        #age was successfully parsed, and we're happy with its value.
        #we're ready to exit the loop.
        break

if age >= 18:
    print("You are able to vote in the United States!")
else:
    print("You are not able to vote in the United States.") 
    
    
def get_non_negative_int(prompt):
    while True:
        try:
            value = int(input(prompt))
        except ValueError:
            print("Sorry, I didn't understand that.")
            continue
        if value < 0:
            print("Sorry, your response must not be negative.")
            continue
        else:
            break
    return value

age = get_non_negative_int("Please enter your wasage: ") 
kids = get_non_negative_int("Please enter the number of children you have: ") 
salary = get_non_negative_int("Please enter your yearly earnings, in dollars: ")
print(age, kids, salary)


def sanitised_input(prompt, type_=None, min_=None, max_=None, range_=None):
    if min_ is not None and max_ is not None and max_ < min_:
        raise ValueError("min_ must be less than or equal to max_.") 
    while True:
        ui = input(prompt)
        if type_ is not None:
            try:
                ui = type_(ui)
            except ValueError:
                print("Input type must be {0}.".format(type_.__name__)) 
                continue 
        if max_ is not None and ui > max_:
            print("Input must be less than or equal to {0}.".format(max_))
        elif min_ is not None and ui < min_:
            print("Input must be greater than or equal to {0}.".format(min_)) 
        elif range_ is not None and ui not in range_:
            if isinstance(range_, range):
                template = "Input must be between {0.start} and {0.stop}."
                print(template.format(range_))
            else:
                template = "Input must be {0}."
                if len(range_) == 1:
                    print(template.format(*range_))
                else:
                    print(template.format(" or ".join((", ".join(map(str, range_[:-1])), str(range_[-1])))))
        else:
            return ui

age = sanitised_input("Enter your fing age: ", int, 1, 101) 
answer = sanitised_input("Enter your answer: ", str.lower, range_=('a', 'b', 'c', 'd'))


In [13]:
'''
Entering the program to do something
'''

while True:
    try:
        print('What do you need to do?')
        data = input("Pick an answer from A to D:")
        if data.lower() not in ('a', 'b', 'c', 'd', 'A', 'B', 'C', 'D'):
            print('Your answer was not correct. \nPlease pick a, b, c, or d.')
            continue
    except ValueError:
        print("Sorry, I didn't understand that.") #better try again... Return to the start of the loop 
        continue
    else:
        break
            

What do you need to do?
Pick an answer from A to D:a


In [32]:
'''
Entering the quarterly forecast
'''
def is_number_true(s):
    """
    Returns True if string is a number.
    """
    try:
        float(s)
        return True
    except ValueError:
        return False


def from_screen():
    count = 0
    while True:
        try:
            print('Enter a forecast in the format noted below (U.S. dollars):')
            print('\t   Fiscal Year, Q1, Q2, Q3, Q4.')
            print('Example:     FY21, 300, 4000, 5050, 6000')
            fscl_year, fq1, fq2, fq3, fq4 = input("Enter AOP:  ").split()
        except ValueError:
            print("Sorry, I didn't understand that.") #better try again... Return to the start of the loop
            continue
        else:
            # break
            return(fscl_year, fq1, fq2, fq3, fq4)



# while True:
#     try:
#         print('Please provide the Fiscal Year Forecast.\n
#         Use the following format in U.S. dollars: Fiscal Year, Q1, Q2, Q3, Q4.\n
#         Here is an illustrative example: FY21 1000000 200000 30000 400.')
#         # data = input("Pick an answer from A to D:")
#         number, fscl_year, fq1, fq2, fq3, fq4 = input(
#                 "Enter AOP:  ").split()
#         if data.lower() not in ('a', 'b', 'c', 'd', 'A', 'B', 'C', 'D'):
#             print('Your answer was not correct. \nPlease pick a, b, c, or d.')
#             continue
#     except ValueError:
#         print("Sorry, I didn't understand that.") #better try again... Return to the start of the loop 
#         continue
#     else:
#         break


data = from_screen()
print(data[1], type(data))

Enter a forecast in the format noted below (U.S. dollars):
	   Fiscal Year, Q1, Q2, Q3, Q4.
Example:     FY21, 300, 4000, 5050, 6000
Enter AOP:  2345 3456 345 3456 634
3456 <class 'tuple'>
